In [1]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Let's look at the training data

In [ ]:
%%bigquery retail_dataset

SELECT
  EXTRACT (dayofweek
  FROM
    date) AS dayofweek,
  EXTRACT (hour
  FROM
    date) AS hour,
  EXTRACT (day
  FROM
    date) AS day,
  EXTRACT (month
  FROM
    date) AS month,
  deviceCategory,
  isMobile,
  operatingSystem,
  browser,
  country,
  city,
  trafficSource,
  trafficMedium,
  trafficCampaign,
  isFirstVisit,
  totalPageviews,
  totalTimeOnSite,
  productPagesViewed,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`
retail_dataset

### Train an DNN model

In [41]:
%%bigquery

CREATE OR REPLACE MODEL
  bqmlretail.dnn_model OPTIONS(MODEL_TYPE='DNN_CLASSIFIER',
    ACTIVATION_FN = 'RELU',
    BATCH_SIZE = 1024,
    DROPOUT = 0.1,
    EARLY_STOP = FALSE,
    HIDDEN_UNITS = [128,
    128,
    128],
    INPUT_LABEL_COLS = ['addedToCart'],
    LEARN_RATE=0.001,
    MAX_ITERATIONS = 50,
    OPTIMIZER = 'ADAGRAD') AS
SELECT
  EXTRACT (dayofweek
  FROM
    date) AS dayofweek,
  EXTRACT (hour
  FROM
    date) AS hour,
  EXTRACT (day
  FROM
    date) AS day,
  EXTRACT (month
  FROM
    date) AS month,
  deviceCategory,
  isMobile,
  operatingSystem,
  browser,
  country,
  city,
  trafficSource,
  trafficMedium,
  trafficCampaign,
  isFirstVisit,
  totalPageviews,
  totalTimeOnSite,
  productPagesViewed,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`;

""


### Get training info

In [4]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL bqmlretail.dnn_model)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,49,0.084303,0.093115,0.001,70826
1,0,48,0.076212,0.093135,0.001,71472
2,0,47,0.089960,0.093153,0.001,73739
3,0,46,0.077185,0.093163,0.001,74874
4,0,45,0.073730,0.093195,0.001,71388
5,0,44,0.078757,0.093208,0.001,71966
6,0,43,0.090668,0.093209,0.001,74952
7,0,42,0.094428,0.093234,0.001,71965
8,0,41,0.095953,0.093257,0.001,80259
9,0,40,0.092167,0.093277,0.001,72613


### Evaluate Model

In [5]:
%%bigquery 
SELECT
  *
FROM
  ML.EVALUATE (MODEL bqmlretail.dnn_model)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.701183,0.442164,0.959559,0.542334,0.093115,0.968622


### Export Model to GCS

In [ ]:
BUCKET = 'gs://<bucket-name>/exported/dnn'

In [44]:
!bq extract --destination_format ML_TF_SAVED_MODEL -m bqmlretail.dnn_model $BUCKET

FATAL Flags parsing error: flag --destination_format=ML_TF_SAVED_MODEL: value should be one of <CSV|NEWLINE_DELIMITED_JSON|AVRO|SAVED_MODEL>
Run 'bq.py help' to get help


In [2]:
!gsutil ls $BUCKET

gs://camus-bqml-retail/exported/dnn/checkpoint
gs://camus-bqml-retail/exported/dnn/graph.pbtxt
gs://camus-bqml-retail/exported/dnn/model.ckpt-18450.data-00000-of-00002
gs://camus-bqml-retail/exported/dnn/model.ckpt-18450.data-00001-of-00002
gs://camus-bqml-retail/exported/dnn/model.ckpt-18450.index
gs://camus-bqml-retail/exported/dnn/model.ckpt-18450.meta
gs://camus-bqml-retail/exported/dnn/saved_model.pb
gs://camus-bqml-retail/exported/dnn/assets/
gs://camus-bqml-retail/exported/dnn/variables/


### Copy model locally

In [9]:
!gsutil cp -r $BUCKET .

/home/jupyter/BQ demo
Copying gs://camus-bqml-retail/exported/dnn/assets/browser.csv...
Copying gs://camus-bqml-retail/exported/dnn/assets/city.csv...                  
Copying gs://camus-bqml-retail/exported/dnn/assets/country.csv...               
Copying gs://camus-bqml-retail/exported/dnn/assets/deviceCategory.csv...        
/ [4 files][  8.5 KiB/  8.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://camus-bqml-retail/exported/dnn/assets/isMobile.csv...
Copying gs://camus-bqml-retail/exported/dnn/assets/operatingSystem.csv...       
Copying gs://camus-bqml-retail/exported/dnn/assets/trafficCampaign.csv...       
Copying gs://camus-bqml-retail/exported/dnn/assets/trafficMedium.csv...         
Copying gs://camus-bqml-re

### Serve in AI Platform

In [36]:
MODEL_DIR=$BUCKET
# Select a suitable version for this model
VERSION_NAME="v1"
FRAMEWORK="TENSORFLOW"

In [37]:
MODEL_NAME="DNN_RETAIL_MODEL"
!gcloud ai-platform models create $MODEL_NAME

ERROR: (gcloud.ai-platform.models.create) Resource in project [camus-retail-demos] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [38]:
!gcloud ai-platform versions create $VERSION_NAME --model=$MODEL_NAME --origin=$MODEL_DIR --runtime-version=1.15 --framework=$FRAMEWORK

ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


In [39]:
!gcloud ai-platform versions describe $VERSION_NAME --model $MODEL_NAME

createTime: '2020-07-10T13:29:19Z'
deploymentUri: gs://camus-bqml-retail/exported/dnn
etag: hbR_4dR4pNk=
framework: TENSORFLOW
isDefault: true
lastUseTime: '2020-07-10T13:33:00Z'
machineType: mls1-c1-m2
name: projects/camus-retail-demos/models/DNN_RETAIL_MODEL/versions/v1
pythonVersion: '2.7'
runtimeVersion: '1.15'
state: READY


### Make a prediction

In [32]:
INPUT_DATA_FILE="test.json"

In [34]:
!gcloud ai-platform predict --model $MODEL_NAME --version $VERSION_NAME --json-instances $INPUT_DATA_FILE

ALL_CLASS_IDS  ALL_CLASSES   CLASS_IDS  CLASSES  LOGISTIC              LOGITS                PROBABILITIES
[0, 1]         [u'1', u'0']  [1]        [u'0']   [0.6921201348304749]  [0.8100497126579285]  [0.3078799247741699, 0.6921201348304749]
